In [14]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from data_storage import create_connection
from stockstats import StockDataFrame

In [15]:
connection = create_connection("../database/crypto_billionairs.db")

In [25]:
table = "BTC_1min_complete_1day_preprocessed_1day_features"

In [26]:
df_temp = pd.read_sql_query(f"select * from {table}", connection)
df_temp = df_temp[-365:].copy()
df_backtesting = pd.DataFrame()
df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_temp[['open', 'high', 'low', 'close', 'volume']]

In [27]:
def init_buy_signal(nothing):
    return df_temp['buy_indicator']

def init_short_signal(nothing):
    return df_temp['short_indicator']

def init_close_long_signal(nothing):
    return df_temp['close_buy_indicator']

def init_close_short_signal(nothing):
    return df_temp['close_short_indicator']
    

In [28]:
class Momentum_AR_raw_long_short(Strategy):
   
    
    def init(self):
        # compute the rsi and stochastic oscillator with stockstats and return the buy signal of the current row
        
        self.init_long_signal = self.I(init_buy_signal, self.data.df)
        self.init_close_long_signal = self.I(init_close_long_signal, self.data.df)
        self.init_short_signal = self.I(init_short_signal, self.data.df)
        self.init_close_short_signal = self.I(init_close_short_signal, self.data.df)

        self.data.df.drop(self.data.df.columns.difference(['Open', 'High', 'Low', 'Close', "Volume"]), 1, inplace=True)
        
       
    
    def next(self):
        
        if self.init_long_signal == 1 and self.position.is_long is False:
            self.position.close()
            self.buy()
            
        elif self.init_close_long_signal == 1:
             self.position.close()
             
        elif self.init_short_signal == -1 and self.position.is_short is False:
            self.position.close()
            self.sell()
            
        elif self.init_close_short_signal == -1:
            self.position.close()
            


In [29]:
bt = Backtest(df_backtesting, Momentum_AR_raw_long_short, cash=100_000, commission=.001)
stats = bt.run()
stats

<ipython-input-29-a85c6ece0b20>:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_backtesting, Momentum_AR_raw_long_short, cash=100_000, commission=.001)


Start                                    2912
End                                      3276
Duration                                  364
Exposure Time [%]                     44.9315
Equity Final [$]                  1.09018e+07
Equity Peak [$]                   1.09111e+07
Return [%]                            10801.8
Buy & Hold Return [%]                -21.5366
Return (Ann.) [%]                           0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                                0
Max. Drawdown [%]                   -0.120824
Avg. Drawdown [%]                  -0.0281125
Max. Drawdown Duration                     10
Avg. Drawdown Duration                  4.325
# Trades                                   85
Win Rate [%]                              100
Best Trade [%]                        14.3263
Worst Trade [%]                       2.64133
Avg. Trade [%]                    

In [8]:
bt.plot()

Row(id='1604', ...)

In [9]:
# %%time 
# stats = bt.optimize(lag=[12, 24, 48, 96, 120, 144, 168],
#                     sell_ = np.arange(60, 90, 3).tolist(),#sell_threshold = np.arange(0, -0.15, -0.01).tolist(),
#                     rsi_lower_bound = np.arange(10, 40, 3).tolist(),
#                     rsi_upper_bound = np.arange(12, 50, 3).tolist(),
#                     #osc_lower_bound = np.arange(10, 90, 5).tolist(),
#                     #osc_upper_bound = np.arange(10, 100, 5).tolist(),
#                     #maximize='Equity Final [$]')#,
#                     constraint= lambda param: param.rsi_lower_bound < param.rsi_upper_bound)
# stats

In [30]:
stats._strategy

<Strategy Momentum_AR_raw_long_short>

In [31]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,5,6,57966.875100,55962.67,2004.205100,0.034575,2917,2918,1
1,1,11,12,59896.716880,63519.00,3622.283120,0.060475,2923,2924,1
2,-1,14,15,63165.810960,61451.85,1713.960960,0.027134,2926,2927,1
3,-1,16,17,59988.771180,56238.60,3750.171180,0.062515,2928,2929,1
4,-1,19,20,56415.528000,53823.48,2592.048000,0.045946,2931,2932,1
...,...,...,...,...,...,...,...,...,...,...
80,-191,336,337,42421.945590,39166.77,621738.537690,0.076733,3248,3249,1
81,225,341,342,38787.909160,41968.00,715520.439000,0.081987,3253,3254,1
82,250,346,347,37816.779000,39678.51,465432.750000,0.049230,3258,3259,1
83,252,348,349,39362.323000,41139.00,447722.604000,0.045136,3260,3261,1


In [12]:
trades = pd.DataFrame(stats['_trades'])

In [13]:
trades.to_excel(f"trades_{table}_short.xlsx")